In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from matplotlib import rc, rcParams
from dataclasses import dataclass

In [2]:
# v = valid traffic, iv = invalid traffic, c = clients, r = readers, or = overwrite, shm = shared memory, disk = logfile, ws = workload stealing
# nr = no regex matching,  

@dataclass
class Params:
    title : str
    ppsv : str
    ppsi : str
    clients : str

paths_fail2ban = {
            'fail2ban_v10k_iv10k_c1016.csv' : Params('Fail2ban',"10k","10k","1016"),
            'fail2ban_v50k_iv1m_c254.csv' : Params('Fail2ban',"10k","1m","254"),
            }

paths_simplefail2ban_socket = {
            'ebpfstats_2024-05-29_02-45-53.csv' : Params('Socket, IPv4', "10k", "10k", "254"),
            'ebpfstats_2024-05-29_02-54-07.csv' : Params('Socket, IPv4', "10k", "1m", "254"),
            'ebpfstats_2024-05-29_03-07-01.csv' : Params('Socket, IPv4 - Rerun', "10k", "1m", "254")
}

def make_time_graph(paths):

    for path, params in paths.items():

        df = pd.read_csv(path,sep=';')

        df_drop = df[df['XDP_action'] == 'XDP_DROP']
        df_pass = df[df['XDP_action'] == 'XDP_PASS']

        fig = plt.figure()
        fig.set_figwidth(20)
        fig.set_figheight(12)
        f_size=25
        rc('text', usetex=True)
        rc('axes', linewidth=1)
        rc('font', weight='bold')
        rc('xtick', labelsize=f_size) 
        rc('ytick', labelsize=f_size) 

        plt.plot(df_pass['time'][::5],df_drop['packets/sec'][::5],label=r'\textbf{Drop}',linewidth=2,color='#ff7f0e')
        plt.plot(df_pass['time'][::5],df_pass['packets/sec'][::5],label=r'\textbf{Pass}',linewidth=2,color='#1f77b4')
        

        title = f'{params.title}: {params.ppsv} wanted PPS, {params.ppsi} unwanted PPS, {params.clients} clients'

        plt.legend(fontsize=f_size, loc='upper right')
        plt.xlabel(r'\textbf{Time [Seconds]}',fontsize=f_size)
        plt.ylabel(r'\textbf{Pakets per second}',fontsize=f_size)
        plt.title(rf'\textbf{{{title}}}',fontsize=f_size)
        plt.grid(axis='y')
        plt.savefig(path.replace('.csv','.png'))


In [ ]:
make_time_graph(paths_simplefail2ban_socket)